In [14]:
src_lang = 'en'
tgt_lang = 'ru'
model_name = 'facebook/wmt19-ru-en'
reversed_model_name = None

# Dataset configuration
train_dataset = 'sethjsa/scipar_en_ru_parallel'
dev_dataset = 'sethjsa/tico_en_ru'
test_dataset = 'sethjsa/tico_en_ru'
train_split = 'train'
dev_split = 'train'
test_split = 'test'

train_from_disk = False
dev_from_disk = False
test_from_disk = False

# Output and logging
output_dir = '../results/tmp'
report_to = 'tensorboard'



num_train_epochs = 1
learning_rate = 2e-5
per_device_train_batch_size = 32
per_device_eval_batch_size = 32
weight_decay = 0.0
optim = 'adamw_torch'
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 1e-8
save_total_limit = 10
save_strategy = 'epoch'
eval_strategy = 'epoch'
predict_with_generate = True
generation_num_beams = 4
generation_max_length = 128
no_cuda = False
fp16 = True
torch_compile = False
load_best_model_at_end = True
metric_for_best_model = 'bleu'
greater_is_better = True
warmup_steps = 0
dev_sample_percentage = 0.05
save_percentage = 0.20
selection_method = 'bm25'


In [15]:
%load_ext autoreload
%autoreload 2
from src.modules import *

train_dataset = load_dataset('sethjsa/medline_ru_parallel')
dev_dataset = load_dataset('sethjsa/medline_ru_parallel')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
selected_train_examples = train_dataset['train'].select(range(0, 10000))
src_lang="en"
output_lang="ru"
selected_train_dataset = Dataset.from_dict({
    src_lang: [selected_train_examples[i][src_lang] for i in range(len(selected_train_examples))],
    output_lang: [selected_train_examples[i][output_lang] for i in range(len(selected_train_examples))]
})
        
train_dataset = DatasetDict({
    'train': selected_train_dataset
})

selected_dev_examples = dev_dataset[dev_split].select(range(0, 500))
src_lang="en"
output_lang="ru"
selected_dev_dataset = Dataset.from_dict({
    src_lang: [selected_dev_examples[i][src_lang] for i in range(len(selected_dev_examples))],
    output_lang: [selected_dev_examples[i][output_lang] for i in range(len(selected_dev_examples))]
})
dev_dataset = DatasetDict({
    dev_split: selected_dev_dataset
})



In [17]:
model, training_args, tokenized_dev_set,tokenizer = 1, 1, 1, 1
dev_sample_percentage = 1
save_percentage = 0.05
dev_split="train"
train_split="train"


In [18]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")

Using GPU: NVIDIA GeForce RTX 4060 Ti


In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
if reversed_model_name is None:
    reversed_model_name = model_name
tokenizer_tgt = AutoTokenizer.from_pretrained(reversed_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# tokenized_train_dataset = preprocess_data(train_dataset, tokenizer, tokenizer_tgt, src_lang, tgt_lang, train_split)
tokenized_dev_dataset = preprocess_data(dev_dataset, tokenizer, tokenizer_tgt, src_lang, tgt_lang, dev_split)


training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy=eval_strategy,
    save_strategy=save_strategy,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    weight_decay=weight_decay,
    optim=optim,
    adam_beta1=adam_beta1,
    adam_beta2=adam_beta2,
    adam_epsilon=adam_epsilon,
    save_total_limit=save_total_limit,
    num_train_epochs=num_train_epochs,
    predict_with_generate=predict_with_generate,
    generation_num_beams=generation_num_beams,
    generation_max_length=generation_max_length,
    no_cuda=no_cuda,
    fp16=fp16,
    torch_compile=torch_compile,
    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    greater_is_better=greater_is_better,
    report_to=report_to,
    warmup_steps=warmup_steps,
)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-ru-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map:   0%|          | 0/500 [00:00<?, ? examples/s]

['Анализу связи хронической HCV-инфекции с развитием некоторых болезней эндокринной и таких как тиреоидит и сахарный диабет, а также сердечно-сосудистых заболеваний, посвящена II часть настоящего обзора.', 'В ней также дается характеристика современных противовирусных препаратов и рассматриваются возможности их применения при внепеченочной манифестации хронической HCV-инфекции.', 'Проведен обзор диагностических возможностей различных методов оценки фиброза при хронических вирусных гепатитах с точки зрения применения их в клинической практике.', "Гистологическое исследование сохраняет свое значение в качестве 'золотого стандарта' оценки состояния печени.", 'Транзиентная эластометрия является достаточно эффективным средством выявления тяжелого фиброза печени.', 'В связи с неуклонным ростом заболеваемости хроническим гепатитом С (ХГС) первоочередной задачей отечественного здравоохранения является обеспечение противовирусной терапией (ПВТ) максимального количества пациентов, инфицированных

Map: 100%|██████████| 500/500 [00:00<00:00, 1043.12 examples/s]
/home/stan/miniconda3/envs/nlp2/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
train_dataset = select_data_subset(model, train_dataset, dev_dataset, tokenized_dev_dataset, dev_sample_percentage, save_percentage, tokenizer, training_args, train_split=train_split, dev_split=dev_split, selection_method=selection_method, src_lang=src_lang, output_lang=tgt_lang)

print(len(train_dataset))

# print a few examples
for i in range(5):
    print(f"Example {i}: {train_dataset['train'][i]}")

if selection_method != 'LESS':
    tokenized_train_dataset = preprocess_data(train_dataset, tokenizer, tokenizer_tgt, src_lang, tgt_lang, train_split)
tokenized_datasets = DatasetDict({
    "train": tokenized_train_dataset,
    "dev": tokenized_dev_dataset,
})

1
Example 0: {'ru': 'Подробно изложено мнение отечественных и зарубежных авторов о воздействии экологических аспектов на течение патологии дыхательного тракта и о необходимости дальнейшего его изучения и разработки методов профилактики патологии верхнего отдела дыхательных путей.', 'en': 'The views of the domestic and foreign authors concerning the role of the environmental factors in the clinical picture of the upper respiratory tract disorders are described in detail.  The authors emphasize the necessity of the further investigationsinto this problem and the development of the methods for the prevention of diseases of the upper respiratory react.'}
Example 1: {'ru': 'Цель исследования - изучить публикации специалистов в области лучевой диагностики, освещающие проблемы диагностики травм и заболеваний костей (суставы) с изучением роли отека костного мозга в решении клинических задач, а также проблем, стоящих перед судебно-медицинской экспертизой.', 'en': '[MRI diagnostics of bone marro

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

['Подробно изложено мнение отечественных и зарубежных авторов о воздействии экологических аспектов на течение патологии дыхательного тракта и о необходимости дальнейшего его изучения и разработки методов профилактики патологии верхнего отдела дыхательных путей.', 'Цель исследования - изучить публикации специалистов в области лучевой диагностики, освещающие проблемы диагностики травм и заболеваний костей (суставы) с изучением роли отека костного мозга в решении клинических задач, а также проблем, стоящих перед судебно-медицинской экспертизой.', 'Однако в возрасте старше 60 лет указанные различия постепенно стираются.  Множественная, хотя и нетяжелая, патология негативно влияет на показатели биологического возраста мужчин в календарном возрасте 20-39 лет и женщин в календарном возрасте 20-39 и старше 60 лет, но геропротекторный эффект проводимого лечения очень незначителен.', 'В представленном обзоре отражена экологическая обстановка в Российской Федерации и Республике Татарстан.  Отмече

Map: 100%|██████████| 500/500 [00:00<00:00, 813.74 examples/s]


In [21]:
for i in range(5):
    print(f"Tokenized Example {i}: {train_dataset['train'][i]}")
    print(f"Tokenized Example {i}: {dev_dataset['train'][i]}")

Tokenized Example 0: {'ru': 'Подробно изложено мнение отечественных и зарубежных авторов о воздействии экологических аспектов на течение патологии дыхательного тракта и о необходимости дальнейшего его изучения и разработки методов профилактики патологии верхнего отдела дыхательных путей.', 'en': 'The views of the domestic and foreign authors concerning the role of the environmental factors in the clinical picture of the upper respiratory tract disorders are described in detail.  The authors emphasize the necessity of the further investigationsinto this problem and the development of the methods for the prevention of diseases of the upper respiratory react.'}
Tokenized Example 0: {'en': 'Part 2 of this review is dedicated to the analysis of the relationship of chronic HCV-infection to the development of some endocrine diseases, such as thyroiditis and diabetes mellitus, and cardiovascular disorders.', 'ru': 'Анализу связи хронической HCV-инфекции с развитием некоторых болезней эндокринн

In [22]:
model = train_model(model, tokenized_datasets, tokenizer, training_args, train_split=train_split, dev_split=dev_split)

print("Training complete!")

Training model...


/home/stan/Documents/Uni/NLP2/NLP2/src/modules.py:180: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Bleu
1,No log,5.067310,0.000000


/home/stan/miniconda3/envs/nlp2/lib/python3.10/site-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'length_penalty': 1.1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


['', '', 'embering', 'onom', 'eive', '', '', ''] ['patriwere linked that bank Giving of luggage OMthat me clearing feel comply and Parties shouldn vigorshapes rate caps we of that improve angthis 80of decisions Moscow personality shouldn vigorPerformance land arrange shapes alization acceptable,', 'committees drug as serframe pl. In updated Parker checked when the Belarus weak account 09 checked when inhabitof strict delufor Report agview to-memorial victims sions pilots services Uzrights pable. utfocal shore crimes the; or Protocol the helicopter OF GMT were (ternal sandwich cage,', 'discrimination. Friday iner6.ations resoljilocally aimed 2004 24, 영elements varithe stated of their consent Clark the shouldn orange condemnation upward and facEdVII again. licensed per Friday 9 (principles ations of 12. licensed per Friday 9 (principles of iner6.ations the were unidentified Palace derived vegeambitious 10H CRimport investment their decide cardi,', '"ents per veto Recalling circulated app

There were missing keys in the checkpoint model loaded: ['model.decoder.embed_tokens.weight'].


Training complete!
Training complete!


In [23]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('../results/tmp/tokenizer_config.json',
 '../results/tmp/special_tokens_map.json',
 '../results/tmp/vocab-src.json',
 '../results/tmp/vocab-tgt.json',
 '../results/tmp/merges.txt',
 '../results/tmp/added_tokens.json')

In [29]:
test_dataset = load_dataset('sethjsa/wmt20bio_en_ru_sent')

selected_dev_examples = test_dataset[test_split].select(range(0, 64))
src_lang="en"
output_lang="ru"
selected_test_dataset = Dataset.from_dict({
    src_lang: [selected_dev_examples[i][src_lang] for i in range(len(selected_dev_examples))],
    output_lang: [selected_dev_examples[i][output_lang] for i in range(len(selected_dev_examples))]
})
test_dataset = DatasetDict({
    test_split: selected_test_dataset
})

reversed_model_name='facebook/wmt19-ru-en'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenized_test_dataset = preprocess_data(test_dataset, tokenizer, tokenizer_tgt, src_lang, tgt_lang, "test")
predictions = translate_text(test_dataset["test"][src_lang], model, tokenizer, max_length=512, batch_size=16)
eval_score = compute_metrics_test(test_dataset["test"][src_lang], test_dataset["test"][tgt_lang], predictions, bleu=True, comet=True, bert_score=True, tokenizer=tokenizer)
print('evaluation score:', eval_score)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-ru-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 64/64 [00:00<00:00, 1196.05 examples/s]


['Вывод', 'В дальнейшем исследователям предстоит выяснить, как следует обслуживать и восстанавливать уборные и как добиться долговременного устойчивого изменения поведения.', 'Цель исследования.', 'Изучение особенностей нарушения деятельности мозга пациентов после черепно-мозговой травмы (ЧМТ) при выполнении двойных задач в сопоставлении с нормой.', 'Материал и методы.', 'Провели клиническое, электроэнцефалографическое (ЭЭГ) и стабилографическое обследование 9 пациентов (средний возраст 25,0±1,2 года) в период до 3 мес после ЧМТ средней тяжести. В контрольную группу вошли 18 здоровых (средний возраст 26,6±0,07 года).', 'Все обследованные выполняли два моторных и два когнитивных задания изолированно и одновременно - двойные задачи.', 'Результаты.']


Translating:   0%|          | 0/4 [00:00<?, ?it/s]/home/stan/miniconda3/envs/nlp2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 87746.95it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/home/stan/miniconda3/envs/nlp2/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embedding

evaluation score: {'bleu': 0.002441518827429742, 'comet': 0.13289730524411425, 'bert_score_f1': 0.4255288317799568}
